Loading the blood donations data

In [44]:
import random
random.seed(100)
import pandas as pd
import sklearn
data=pd.read_csv('C:/Users/sadhv/OneDrive/Desktop/technocolabs/transfusion.csv')
data.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


Inspecting transfusion DataFrame

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 5 columns):
 #   Column                                      Non-Null Count  Dtype
---  ------                                      --------------  -----
 0   Recency (months)                            748 non-null    int64
 1   Frequency (times)                           748 non-null    int64
 2   Monetary (c.c. blood)                       748 non-null    int64
 3   Time (months)                               748 non-null    int64
 4   whether he/she donated blood in March 2007  748 non-null    int64
dtypes: int64(5)
memory usage: 29.3 KB


In [15]:
data.describe()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
count,748.000000,748.000000,748.000000,748.000000,748.000000
mean,9.506684,5.514706,1378.676471,34.282086,0.237968
std,8.095396,5.839307,1459.826781,24.376714,0.426124
min,0.000000,1.000000,250.000000,2.000000,0.000000
25%,2.750000,2.000000,500.000000,16.000000,0.000000
50%,7.000000,4.000000,1000.000000,28.000000,0.000000
75%,14.000000,7.000000,1750.000000,50.000000,0.000000
max,74.000000,50.000000,12500.000000,98.000000,1.000000


Creating target column

In [16]:
target=data['whether he/she donated blood in March 2007']
data=data.drop(['whether he/she donated blood in March 2007'],axis=1)
target.value_counts()

0    570
1    178
Name: whether he/she donated blood in March 2007, dtype: int64

Checking target incidence

In [17]:
target.value_counts(normalize=True)*100

0    76.203209
1    23.796791
Name: whether he/she donated blood in March 2007, dtype: float64

Splitting transfusion into train and test datasets


In [20]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(data,target,stratify=target,test_size=0.3,random_state=42)
Y_train.value_counts(normalize=True)*100

0    76.290631
1    23.709369
Name: whether he/she donated blood in March 2007, dtype: float64

In [21]:
Y_test.value_counts(normalize=True)*100

0    76.0
1    24.0
Name: whether he/she donated blood in March 2007, dtype: float64

Selecting model using TPOT

In [30]:
from tpot import TPOTClassifier
from sklearn.metrics import roc_auc_score
model=TPOTClassifier(generations=5,population_size=20,scoring='roc_auc',verbosity=2)
model.fit(X_train,Y_train.values.ravel())
model.export('Bloodtransfusion.py')


Generation 1 - Current best internal CV score: 0.7384327267932489

Generation 2 - Current best internal CV score: 0.7384327267932489

Generation 3 - Current best internal CV score: 0.7384327267932489

Generation 4 - Current best internal CV score: 0.7384327267932489

Generation 5 - Current best internal CV score: 0.7400098892405064

Best pipeline: LogisticRegression(MultinomialNB(input_matrix, alpha=0.1, fit_prior=True), C=0.1, dual=False, penalty=l2)


In [34]:
from sklearn.metrics import accuracy_score
Y_pred=model.predict(X_test)
print("\nAUC score:%f"%(roc_auc_score(Y_test,Y_pred)*100))
print("\nAccuracy score:%f"%(accuracy_score(Y_test,Y_pred)*100))


AUC score:55.019493

Accuracy score:76.888889


Checking the variance

In [35]:
data.var()

Recency (months)         6.553543e+01
Frequency (times)        3.409751e+01
Monetary (c.c. blood)    2.131094e+06
Time (months)            5.942242e+02
dtype: float64

 Log normalization

In [37]:
import numpy as np
data['LogNormalMonetary']=np.log(data['Monetary (c.c. blood)'])
data=data.drop(['Monetary (c.c. blood)'],axis=1)
data.head()

,Recency (months),Frequency (times),Time (months),LogNormalMonetary
0,2,50,98,9.433484
1,0,13,28,8.086410
2,1,16,35,8.294050
3,2,20,45,8.517193
4,1,24,77,8.699515


In [38]:
data.var()

Recency (months)      65.535430
Frequency (times)     34.097508
Time (months)        594.224203
LogNormalMonetary      0.836348
dtype: float64

Training the logistic regression model

In [39]:
X_train,X_test,Y_train,Y_test=train_test_split(data,target,test_size=0.3,stratify=target)
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train,Y_train.values.ravel())
Y_pred=lr.predict(X_test)
Y_pred

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0], dtype=int64)

In [40]:
print(Y_test.values)

[0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0
 0 0 0 1 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 1 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1
 1 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1
 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 1 0 1 0 0 0 1 0 0
 0 0 0]


Conclusion

In [43]:
from sklearn.metrics import confusion_matrix
print("\nAUC score:%f"%(roc_auc_score(Y_test,Y_pred)*100))
print("\naccuracy score:%f"%(accuracy_score(Y_test,Y_pred)*100))
print(confusion_matrix(Y_test,Y_pred))


AUC score:62.719298

accuracy score:80.888889
[[167   4]
 [ 39  15]]
